In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from pymongo import MongoClient
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import pandas as pd

In [2]:
client = MongoClient('localhost', 27017)
db = client.TFE
collection = db.TESTS

In [3]:
def generateFromCM(cm):
    tn, fp, fn, tp = cm
    y_true = ['reliable' for i in range(0, tn + fp)]
    y_pred = ['reliable' for i in range(0, tn)]+['fake' for i in range(0, fp)]
    y_true = y_true + ['fake' for i in range(0, fn + tp)]
    y_pred = y_pred + ['reliable' for i in range(0, fn)] + ['fake' for i in range(0, tp)]
    return y_true, y_pred

In [4]:
cursor = collection.find({'experiment_id' : 1, 'finish' : True}, no_cursor_timeout=True)
pbar = tqdm(total=1)
for i, res in enumerate(cursor):
    for i, result in enumerate(res['result']):
        epoch = result['epoch']
        cm = result['valid']['Confusion Matrix']
        y_true, y_pred = generateFromCM(cm)
        clr = classification_report(y_true, y_pred, output_dict = True)
        collection.update_one({'experiment_id' : res['experiment_id'], 'result.epoch' : epoch}, {"$set" : {'result.$.valid.clr' : clr}})
        
        cm = result['train']['Confusion Matrix']
        y_true, y_pred = generateFromCM(cm)
        clr = classification_report(y_true, y_pred, output_dict = True)
        collection.update_one({'experiment_id' : res['experiment_id'], 'result.epoch' : epoch}, {"$set" : {'result.$.train.clr' : clr}})
    pbar.update(1)
pbar.close()
cursor.close()

100%|██████████| 1/1 [01:36<00:00, 96.52s/it]


In [8]:
clr

{'fake': {'precision': 0.9239253483092063,
  'recall': 0.9339965220949263,
  'f1-score': 0.9289336389677527,
  'support': 156416},
 'reliable': {'precision': 0.9332691275992011,
  'recall': 0.923096102700491,
  'f1-score': 0.9281547406686016,
  'support': 156416},
 'accuracy': 0.9285463123977087,
 'macro avg': {'precision': 0.9285972379542037,
  'recall': 0.9285463123977087,
  'f1-score': 0.9285441898181772,
  'support': 312832},
 'weighted avg': {'precision': 0.9285972379542037,
  'recall': 0.9285463123977087,
  'f1-score': 0.9285441898181771,
  'support': 312832}}

In [10]:
y_true, y_pred = generateFromCM([1032,  396,  590,  516])

In [15]:
clr = classification_report(y_true, y_pred, output_dict = True)

In [16]:
pd.DataFrame(clr).to_latex('out.tex')

In [5]:
y_true, y_pred = generateFromCM([914, 514, 592, 514])
clr = classification_report(y_true, y_pred, output_dict = True)
pd.DataFrame(clr).to_latex('out.tex')

In [7]:
pd.DataFrame(clr)

,fake,reliable,accuracy,macro avg,weighted avg
f1-score,0.481724,0.623040,0.563536,0.552382,0.561361
precision,0.500000,0.606906,0.563536,0.553453,0.560245
recall,0.464738,0.640056,0.563536,0.552397,0.563536
support,1106.000000,1428.000000,0.563536,2534.000000,2534.000000


In [8]:
y_true, y_pred = generateFromCM([892, 536, 578, 528])
clr = classification_report(y_true, y_pred, output_dict = True)
pd.DataFrame(clr).to_latex('out.tex')

In [9]:
pd.DataFrame(clr)

,fake,reliable,accuracy,macro avg,weighted avg
f1-score,0.486636,0.615597,0.560379,0.551116,0.559310
precision,0.496241,0.606803,0.560379,0.551522,0.558546
recall,0.477396,0.624650,0.560379,0.551023,0.560379
support,1106.000000,1428.000000,0.560379,2534.000000,2534.000000


In [12]:
y_true, y_pred = generateFromCM([1012,  416,  676,  430])
clr = classification_report(y_true, y_pred, output_dict = True)
df = pd.DataFrame(clr)

In [13]:
df.to_latex('out.tex')

In [14]:
df

,fake,reliable,accuracy,macro avg,weighted avg
f1-score,0.440574,0.649551,0.569061,0.545062,0.558340
precision,0.508274,0.599526,0.569061,0.553900,0.559698
recall,0.388788,0.708683,0.569061,0.548736,0.569061
support,1106.000000,1428.000000,0.569061,2534.000000,2534.000000
